In [1]:
import cv2
import numpy as np
from PanelControl import *

In [3]:
def empty(a) :
    pass

cv2.namedWindow("Bars", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Bars", 500, 300)
# Create trackbar agregar por defecto el 0
cv2.createTrackbar("Contrast", "Bars", 100, 300, empty) # 1 (no cambia el contraste) [0 255]
cv2.createTrackbar("Brightness", "Bars", 100, 200, empty) # 0 (no cambia el brillo). Rango: [-255, 255]
cv2.createTrackbar("Gamma", "Bars", 100, 300, empty) # 1 (no tiene efecto) 

# Videos Equalizado
cv2.createTrackbar("Contrast Eq", "Bars", 100, 300, empty) # 1 (no cambia el contraste) [0 255]
cv2.createTrackbar("Brightness Eq", "Bars", 100, 200, empty) # 0 (no cambia el brillo). Rango: [-255, 255]
cv2.createTrackbar("Gamma Eq", "Bars", 100, 300, empty) # 1 (no tiene efecto) 

cv2.createTrackbar("Clip", "Bars", 100, 200, empty) #
cv2.createTrackbar("Grid size", "Bars", 0, 16, empty) #  

# Cargar imagen original solo una vez
# img = cv2.imread("./darkPollen.jpg")

vid = cv2.VideoCapture(0)

while True :
    # Lectura de video
    _, frame = vid.read()

    contraste  = cv2.getTrackbarPos("Contrast", "Bars") / 100
    contraste = max(contraste, 0.1) # El inicio es en (0.1)

    brillo = cv2.getTrackbarPos("Brightness", "Bars") - 100

    gamma = cv2.getTrackbarPos("Gamma", "Bars") / 100
    gamma = max(gamma, 0.1)

    # Controles vid eq.
    contraste_eq  = cv2.getTrackbarPos("Contrast Eq", "Bars") / 100
    contraste_eq = max(contraste_eq, 0.1) # El inicio es en (0.1)

    brillo_eq = cv2.getTrackbarPos("Brightness Eq", "Bars") - 100

    gamma_eq = cv2.getTrackbarPos("Gamma Eq", "Bars") / 100
    gamma_eq = max(gamma_eq, 0.1)

    clip = cv2.getTrackbarPos("Clip", "Bars") / 100
    tilegrid = cv2.getTrackbarPos("Grid size", "Bars") + 8

    # Ajuste de contraste
    frame_contraste = cv2.convertScaleAbs(frame, alpha=contraste, beta=0)
    frame_contraste = np.clip(frame_contraste, 0, 255).astype(np.uint8)

    # Ajuste de brillo
    frame_brillo = cv2.add(frame_contraste, (brillo, brillo, brillo))
    frame_brillo = np.clip(frame_brillo, 0, 255).astype(np.uint8)

    # Ajuste gamma
    frame_gamma = np.power(frame_brillo/255, gamma)*255
    frame_gamma = np.clip(frame_gamma, 0, 255).astype(np.uint8)

    clahe = cv2.createCLAHE( clipLimit=clip, tileGridSize=(tilegrid, tilegrid))

    # Frame ecualizado

    frame_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    frame_hsv[:, :, 2] = clahe.apply(frame_hsv[:, :, 2])
    frame_hsv_bgr = cv2.cvtColor(frame_hsv, cv2.COLOR_HSV2BGR)

    fr_contraste_eq = cv2.convertScaleAbs(frame_hsv_bgr, alpha=contraste_eq, beta=0)
    fr_contraste_eq = np.clip(fr_contraste_eq, 0, 255).astype(np.uint8)

    # Ajuste de brillo
    fr_brillo_eq = cv2.add(fr_contraste_eq, (brillo_eq, brillo_eq, brillo_eq))
    fr_brillo_eq = np.clip(fr_brillo_eq, 0, 255).astype(np.uint8)

    # Ajuste gamma
    fr_gamma_eq = np.power(fr_brillo_eq/255, gamma_eq)*255
    fr_gamma_eq = np.clip(fr_gamma_eq, 0, 255).astype(np.uint8)



    config = {
        'imagenes' : [frame, frame_contraste, frame_brillo, frame_gamma, frame_hsv_bgr, fr_contraste_eq, fr_brillo_eq, fr_gamma_eq],
        'etiquetas' : ["Original", "Frame Contraste", "Frame Brillo", "Frame Gamma", "Frame Ecualizado", "Frame Eq. Contraste", "Frame Eq. Brillo", "Frame Eq. Gamma"],
        'filas': 2,
        'columnas': 4,
        'tamaño_base' : 250,
        'escala_panel' : 100
    }

    panel = crear_panel(**config)
    cv2.imshow('Panel', panel)

    if cv2.waitKey(1) & 0xFF == 27 :
        break

vid.release()
cv2.destroyAllWindows()

